# 04. 캡슐화 (Encapsulation)

## 학습 목표
- 캡슐화의 개념과 중요성을 이해한다
- 접근 제어자 사용법을 익힌다
- @property 데코레이터를 활용한다
- 안전한 데이터 관리 방법을 습득한다

---

## 1. 캡슐화란?

**캡슐화(Encapsulation)**는 객체의 내부 데이터와 메서드를 외부에서 직접 접근하지 못하도록 숨기는 것입니다.

### 캡슐화의 장점:
1. **데이터 보호**: 잘못된 값이 설정되는 것을 방지
2. **내부 구조 은닉**: 클래스 내부 구현을 외부에서 알 필요 없음
3. **유지보수 향상**: 내부 구현 변경 시 외부 코드에 영향 최소화
4. **일관성 유지**: 정해진 규칙에 따른 데이터 접근

In [ ]:
# 캡슐화 없는 클래스 (문제점 예시)
class BadBankAccount:
    def __init__(self, account_number, initial_balance=0):
        self.account_number = account_number
        self.balance = initial_balance  # 모든 속성이 공개
        self.pin = "1234"

# 문제점 발생
bad_account = BadBankAccount("123-456", 100000)
print(f"초기 잔액: {bad_account.balance:,}원")

# 직접 접근으로 인한 문제들
bad_account.balance = -50000  # 음수 잔액 설정 가능
print(f"문제 상황 1 - 음수 잔액: {bad_account.balance:,}원")

bad_account.pin = "wrong"  # PIN을 임의로 변경 가능
print(f"문제 상황 2 - PIN 노출: {bad_account.pin}")

bad_account.balance = "잔액"  # 잘못된 타입 설정 가능
print(f"문제 상황 3 - 잘못된 타입: {bad_account.balance}")

## 2. 접근 제어자의 종류

파이썬에서는 언더스코어(_)를 사용해서 속성이나 메서드의 접근 수준을 표현해  
- **public**: 아무 표시 없이 작성 (외부에서 자유롭게 접근 가능)  
- **protected**: 이름 앞에 언더스코어 하나(`_`)를 붙여  
  이건 "이 속성은 내부적으로만(즉, 클래스 내부나 상속받은 자식 클래스에서만) 사용하길 권장한다"는 의미입니다.  
  실제로 막는 건 아니고, 개발자끼리의 약속 같은 것으로  
  자바에서 protected는 상속받은 클래스에서만 접근 가능하게 제한하는데,  
  파이썬의 `_`는 그런 강제성은 없지만, 상속받은 클래스에서는 접근해서 쓸 수 있다는 점에서 비슷한 점이 있습니다.
- **private**: 이름 앞에 언더스코어 두 개(`__`)를 붙여  
  이건 외부에서 직접 접근하지 못하도록  
  파이썬이 변수 이름을 내부적으로 바꿔주는 기능이 적용돼  
  이걸 '네임 맹글링(name mangling)'이라고 해  
  여기서 'mangle'이라는 단어는 '뒤섞다', '엉망으로 만들다'라는 뜻이야  
  즉, 변수 이름 앞에 클래스 이름이 자동으로 붙어서  
  예를 들어 `__pin`은 실제로는 `_클래스이름__pin`처럼 바뀌어 저장돼  
  그래서 실수로 외부에서 접근하거나 변경하는 걸 방지해주는 역할을 해  
  완벽하게 막는 건 아니지만, 의도치 않은 접근을 어렵게 만들어주는 장치라고 보면 돼  

In [1]:
# 캡슐화를 적용한 은행 계좌 클래스
class BankAccount:
    def __init__(self, account_number, initial_balance=0):
        self.account_number = account_number    # 공개 속성
        self._balance = initial_balance         # 보호된 속성
        self.__pin = "1234"                    # 비공개 속성
        self._transaction_count = 0             # 보호된 속성
    
    # 잔액 조회 메서드 (안전한 접근)
    def get_balance(self):
        return self._balance
    
    # 입금 메서드
    def deposit(self, amount):
        if amount <= 0:
            print("입금 금액은 0보다 커야 합니다.")
            return False
        
        self._balance += amount
        self._transaction_count += 1
        print(f"{amount:,}원 입금 완료. 현재 잔액: {self._balance:,}원")
        return True
    
    # 출금 메서드
    def withdraw(self, amount, pin):
        # PIN 검증
        if not self.__verify_pin(pin):
            print("PIN이 올바르지 않습니다.")
            return False
        
        # 금액 검증
        if amount <= 0:
            print("출금 금액은 0보다 커야 합니다.")
            return False
        
        if amount > self._balance:
            print("잔액이 부족합니다.")
            return False
        
        self._balance -= amount
        self._transaction_count += 1
        print(f"{amount:,}원 출금 완료. 현재 잔액: {self._balance:,}원")
        return True
    
    # 비공개 메서드 - PIN 확인
    def __verify_pin(self, pin):
        return pin == self.__pin
    
    # PIN 변경 메서드
    def change_pin(self, old_pin, new_pin):
        if not self.__verify_pin(old_pin):
            print("기존 PIN이 올바르지 않습니다.")
            return False
        
        if len(new_pin) != 4 or not new_pin.isdigit():
            print("PIN은 4자리 숫자여야 합니다.")
            return False
        
        self.__pin = new_pin
        print("PIN이 성공적으로 변경되었습니다.")
        return True
    
    # 거래 내역 조회
    def get_transaction_count(self):
        return self._transaction_count

# 안전한 계좌 사용
print("=== 캡슐화 적용된 계좌 ===")
account = BankAccount("123-456-789", 100000)

# 공개 속성 접근
print(f"계좌번호: {account.account_number}")
print(f"현재 잔액: {account.get_balance():,}원")

# 안전한 거래
account.deposit(50000)
account.withdraw(30000, "1234")
account.withdraw(30000, "wrong")

print(f"총 거래 횟수: {account.get_transaction_count()}회")

=== 캡슐화 적용된 계좌 ===
계좌번호: 123-456-789
현재 잔액: 100,000원
50,000원 입금 완료. 현재 잔액: 150,000원
30,000원 출금 완료. 현재 잔액: 120,000원
PIN이 올바르지 않습니다.
총 거래 횟수: 2회


## 3. 접근 제어 테스트

In [2]:
print("=== 접근 제어 테스트 ===")

# 1. 공개 속성 접근 - 성공
print(f"공개 속성 접근: {account.account_number}")

# 2. 보호된 속성 접근 - 가능하지만 권장하지 않음
print(f"보호된 속성 접근 (권장 안함): {account._balance:,}원")

# 3. 비공개 속성 직접 접근 시도 - 오류 발생
try:
    print(account.__pin)
except AttributeError as e:
    print(f"비공개 속성 접근 실패: {e}")

# 4. Name mangling을 통한 접근 (권장하지 않음)
print(f"Name mangling 접근 (권장 안함): {account._BankAccount__pin}")

# 5. 직접 수정 시도 (권장하지 않음)
account._balance = -1000000  # 보호된 속성 직접 수정
print(f"직접 수정 후 잔액: {account.get_balance():,}원")

# 원래 상태로 복구
account._balance = 120000

=== 접근 제어 테스트 ===
공개 속성 접근: 123-456-789
보호된 속성 접근 (권장 안함): 120,000원
비공개 속성 접근 실패: 'BankAccount' object has no attribute '__pin'
Name mangling 접근 (권장 안함): 1234
직접 수정 후 잔액: -1,000,000원


## 4. @property 데코레이터

@property, @속성명.setter, @속성명.getter의 차이점은 실제 사용 목적과 상황에 따라 구분된다.  
   
1. @property는 읽기 전용 속성을 정의할 때 사용한다.  
   즉, 외부에서 속성처럼 값을 읽을 수 있도록 하되, 내부적으로는 메서드로 동작하게 하여  
   캡슐화와 데이터 보호를 동시에 달성할 수 있다.  
   예를 들어, 계산된 값이나 내부 상태를 외부에 노출할 때 사용한다.  
   
2. @속성명.setter는 해당 속성에 값을 쓸 수 있도록 허용할 때 사용한다.  
   setter를 정의하면, 속성에 값을 할당할 때 내부적으로 유효성 검사나 추가 로직을 실행할 수 있다.  
   즉, 단순히 값을 저장하는 것이 아니라, 값의 유효성, 타입 검사, 부수 효과 등을 제어하고 싶을 때 setter를 사용한다.  
   setter가 정의되지 않은 경우, 해당 속성은 읽기 전용이 된다.  
   
3. @속성명.getter는 이미 property로 정의된 속성의 getter를  
   상속받은 클래스에서 별도로 재정의하고 싶을 때 주로 사용한다.  
   즉, 부모 클래스에서 정의된 property의 getter만 변경하고,  
   setter 등 다른 접근자는 그대로 두고 싶을 때 활용한다.  
   일반적인 상황에서는 @property로 getter를 정의하는 것이 충분하며,  
   상속 구조에서 getter만 오버라이드할 필요가 있을 때 @속성명.getter를 사용한다.  
   
따라서,  
- 단순히 읽기 전용 속성을 만들고 싶을 때는 @property만 사용하면 되고,  
- 읽기와 쓰기 모두 제어하고 싶을 때는 @property와 @속성명.setter를 함께 사용한다.  
- 상속 등에서 getter만 별도로 재정의해야 할 때 @속성명.getter를 사용한다.  
   
이처럼 각 데코레이터는 목적과 사용 상황에 따라 선택적으로 사용해야 하며,  
단순히 문법적 차이만이 아니라 실제 캡슐화와 데이터 보호, 상속 구조에서의 유연성 등  
객체지향 설계의 의도에 맞게 활용하는 것이 중요하다.  





In [3]:
class Temperature:
    def __init__(self, celsius=0):
        self._celsius = celsius
    
    @property
    def celsius(self):
        """섭씨 온도 getter"""
        return self._celsius
    
    @celsius.setter
    def celsius(self, value):
        """섭씨 온도 setter (유효성 검사 포함)"""
        if not isinstance(value, (int, float)):
            raise TypeError("온도는 숫자여야 합니다.")
        if value < -273.15:
            raise ValueError("절대영도(-273.15°C) 이하는 불가능합니다.")
        if value > 1000:
            raise ValueError("온도가 너무 높습니다.")
        self._celsius = value
    
    @property
    def fahrenheit(self):
        """화씨 온도 getter (계산된 값)"""
        return (self._celsius * 9/5) + 32
    
    @fahrenheit.setter
    def fahrenheit(self, value):
        """화씨 온도 setter (섭씨로 변환 후 저장)"""
        celsius_value = (value - 32) * 5/9
        self.celsius = celsius_value  # 섭씨 setter를 통해 유효성 검사
    
    @property
    def kelvin(self):
        """켈빈 온도 getter (읽기 전용)"""
        return self._celsius + 273.15
    
    @property
    def description(self):
        """온도 상태 설명"""
        if self._celsius < 0:
            return "빙점 이하 (얼음)"
        elif self._celsius == 0:
            return "빙점 (물/얼음)"
        elif self._celsius < 100:
            return "액체 상태 (물)"
        elif self._celsius == 100:
            return "끓는점 (물/수증기)"
        else:
            return "끓는점 이상 (수증기)"
    
    def __str__(self):
        return f"{self._celsius}°C ({self.fahrenheit:.1f}°F, {self.kelvin:.1f}K) - {self.description}"

# 온도 클래스 테스트
print("=== @property 데코레이터 예제 ===")
temp = Temperature(25)
print(f"초기 온도: {temp}")

# 속성처럼 접근 (실제로는 메서드)
print(f"\n개별 값 접근:")
print(f"섭씨: {temp.celsius}°C")
print(f"화씨: {temp.fahrenheit:.1f}°F")
print(f"켈빈: {temp.kelvin:.1f}K")
print(f"상태: {temp.description}")

# setter 사용
print(f"\n온도 변경 테스트:")
temp.celsius = 0
print(f"0도로 설정: {temp}")

temp.fahrenheit = 212  # 화씨 212도 (섭씨 100도)
print(f"화씨 212도로 설정: {temp}")

# 유효성 검사 테스트
print(f"\n유효성 검사 테스트:")
test_values = [-300, "문자열", 1500]

for value in test_values:
    try:
        temp.celsius = value
        print(f"{value} 설정 성공")
    except (ValueError, TypeError) as e:
        print(f"{value} 설정 실패: {e}")

=== @property 데코레이터 예제 ===
초기 온도: 25°C (77.0°F, 298.1K) - 액체 상태 (물)

개별 값 접근:
섭씨: 25°C
화씨: 77.0°F
켈빈: 298.1K
상태: 액체 상태 (물)

온도 변경 테스트:
0도로 설정: 0°C (32.0°F, 273.1K) - 빙점 (물/얼음)
화씨 212도로 설정: 100.0°C (212.0°F, 373.1K) - 끓는점 (물/수증기)

유효성 검사 테스트:
-300 설정 실패: 절대영도(-273.15°C) 이하는 불가능합니다.
문자열 설정 실패: 온도는 숫자여야 합니다.
1500 설정 실패: 온도가 너무 높습니다.


## 5. 실용적인 캡슐화 예제 - 학생 성적 관리

In [ ]:
class Student:
    def __init__(self, student_id, name):
        self._student_id = student_id
        self._name = name
        self._scores = {}  # 과목별 점수 저장
        self._attendance = 0  # 출석 일수
        self.__password = "default123"  # 비밀번호
    
    @property
    def student_id(self):
        return self._student_id
    
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, value):
        if not isinstance(value, str) or len(value.strip()) == 0:
            raise ValueError("이름은 빈 문자열이 될 수 없습니다.")
        self._name = value.strip()
    
    @property
    def attendance(self):
        return self._attendance
    
    def add_score(self, subject, score, password):
        """성적 추가 (비밀번호 확인 필요)"""
        if not self.__verify_password(password):
            print("비밀번호가 올바르지 않습니다.")
            return False
        
        if not isinstance(score, (int, float)) or not (0 <= score <= 100):
            print("점수는 0-100 사이의 숫자여야 합니다.")
            return False
        
        self._scores[subject] = score
        print(f"{subject} 점수 {score}점이 추가되었습니다.")
        return True
    
    def get_score(self, subject):
        """특정 과목 점수 조회"""
        return self._scores.get(subject, None)
    
    def get_all_scores(self):
        """모든 점수 조회 (복사본 반환)"""
        return self._scores.copy()
    
    @property
    def average_score(self):
        """평균 점수 계산"""
        if not self._scores:
            return 0
        return sum(self._scores.values()) / len(self._scores)
    
    @property
    def grade(self):
        """학점 계산"""
        avg = self.average_score
        if avg >= 90:
            return 'A'
        elif avg >= 80:
            return 'B'
        elif avg >= 70:
            return 'C'
        elif avg >= 60:
            return 'D'
        else:
            return 'F'
    
    def add_attendance(self, days=1):
        """출석 일수 추가"""
        if days < 0:
            print("출석 일수는 음수가 될 수 없습니다.")
            return False
        self._attendance += days
        print(f"출석 {days}일 추가. 총 출석: {self._attendance}일")
        return True
    
    def change_password(self, old_password, new_password):
        """비밀번호 변경"""
        if not self.__verify_password(old_password):
            print("기존 비밀번호가 올바르지 않습니다.")
            return False
        
        if len(new_password) < 6:
            print("새 비밀번호는 6자 이상이어야 합니다.")
            return False
        
        self.__password = new_password
        print("비밀번호가 성공적으로 변경되었습니다.")
        return True
    
    def __verify_password(self, password):
        """비밀번호 확인 (비공개 메서드)"""
        return password == self.__password
    
    def get_summary(self):
        """학생 정보 요약"""
        return {
            'student_id': self.student_id,
            'name': self.name,
            'subjects_count': len(self._scores),
            'average_score': round(self.average_score, 2),
            'grade': self.grade,
            'attendance': self.attendance
        }
    
    def __str__(self):
        return f"학생 {self.name}({self.student_id}) - 평균: {self.average_score:.1f}점 ({self.grade}학점)"

# 학생 성적 관리 시스템 테스트
print("=== 학생 성적 관리 시스템 ===")
student = Student("2023001", "김학생")

# 초기 정보
print(f"등록된 학생: {student}")
print(f"초기 요약: {student.get_summary()}")

# 성적 추가
print(f"\n=== 성적 추가 ===")
student.add_score("수학", 85, "default123")
student.add_score("영어", 92, "default123")
student.add_score("과학", 78, "default123")
student.add_score("국어", 88, "wrong_password")  # 잘못된 비밀번호
student.add_score("국어", 88, "default123")  # 올바른 비밀번호

# 출석 추가
print(f"\n=== 출석 관리 ===")
student.add_attendance(5)
student.add_attendance(3)

# 최종 결과
print(f"\n=== 최종 결과 ===")
print(f"학생 정보: {student}")
print(f"전체 점수: {student.get_all_scores()}")
print(f"상세 요약: {student.get_summary()}")

# 비밀번호 변경
print(f"\n=== 비밀번호 변경 ===")
student.change_password("wrong", "new123456")  # 실패
student.change_password("default123", "new123456")  # 성공

## 6. 실습 문제

In [ ]:
# 실습 문제: 도서 관리 시스템
# Book 클래스를 구현하세요.
# 요구사항:
# 1. 비공개 속성: __isbn, __title, __author, __available
# 2. @property를 사용한 안전한 접근
# 3. 대출/반납 기능 (비밀번호 확인 필요)
# 4. 유효성 검사 포함

class Book:
    def __init__(self, isbn, title, author, admin_password="admin123"):
        # 입력 유효성 검사
        if not isbn or not title or not author:
            raise ValueError("ISBN, 제목, 저자는 빈 값일 수 없습니다.")
        
        self.__isbn = isbn
        self.__title = title
        self.__author = author
        self.__available = True
        self.__borrowed_by = None
        self.__admin_password = admin_password
        self.__borrow_count = 0
    
    @property
    def isbn(self):
        return self.__isbn
    
    @property
    def title(self):
        return self.__title
    
    @title.setter
    def title(self, value):
        if not value or not value.strip():
            raise ValueError("제목은 빈 값일 수 없습니다.")
        self.__title = value.strip()
    
    @property
    def author(self):
        return self.__author
    
    @author.setter
    def author(self, value):
        if not value or not value.strip():
            raise ValueError("저자는 빈 값일 수 없습니다.")
        self.__author = value.strip()
    
    @property
    def is_available(self):
        return self.__available
    
    @property
    def borrowed_by(self):
        return self.__borrowed_by
    
    @property
    def borrow_count(self):
        return self.__borrow_count
    
    def borrow(self, borrower_name, admin_password):
        """도서 대출"""
        if not self.__verify_admin(admin_password):
            print("관리자 인증이 필요합니다.")
            return False
        
        if not self.__available:
            print(f"'{self.__title}'은(는) 이미 대출 중입니다. (대출자: {self.__borrowed_by})")
            return False
        
        if not borrower_name or not borrower_name.strip():
            print("대출자 이름을 입력해주세요.")
            return False
        
        self.__available = False
        self.__borrowed_by = borrower_name.strip()
        self.__borrow_count += 1
        print(f"'{self.__title}'이(가) {self.__borrowed_by}님에게 대출되었습니다.")
        return True
    
    def return_book(self, admin_password):
        """도서 반납"""
        if not self.__verify_admin(admin_password):
            print("관리자 인증이 필요합니다.")
            return False
        
        if self.__available:
            print(f"'{self.__title}'은(는) 이미 반납된 상태입니다.")
            return False
        
        returned_by = self.__borrowed_by
        self.__available = True
        self.__borrowed_by = None
        print(f"'{self.__title}'이(가) {returned_by}님으로부터 반납되었습니다.")
        return True
    
    def change_admin_password(self, old_password, new_password):
        """관리자 비밀번호 변경"""
        if not self.__verify_admin(old_password):
            print("기존 관리자 비밀번호가 올바르지 않습니다.")
            return False
        
        if len(new_password) < 6:
            print("새 비밀번호는 6자 이상이어야 합니다.")
            return False
        
        self.__admin_password = new_password
        print("관리자 비밀번호가 성공적으로 변경되었습니다.")
        return True
    
    def __verify_admin(self, password):
        """관리자 비밀번호 확인 (비공개 메서드)"""
        return password == self.__admin_password
    
    def get_info(self):
        """도서 정보 반환"""
        status = "대출 가능" if self.__available else f"대출 중 ({self.__borrowed_by})"
        return {
            'isbn': self.isbn,
            'title': self.title,
            'author': self.author,
            'status': status,
            'borrow_count': self.borrow_count
        }
    
    def __str__(self):
        status = "[대출가능]" if self.__available else f"[대출중-{self.__borrowed_by}]"
        return f"{status} '{self.__title}' - {self.__author} (ISBN: {self.__isbn})"

# 도서 관리 시스템 테스트
print("=== 도서 관리 시스템 테스트 ===")

# 도서 생성
books = [
    Book("978-1234567890", "파이썬 프로그래밍", "김파이"),
    Book("978-0987654321", "자료구조와 알고리즘", "이구조"),
    Book("978-1111111111", "웹 개발 입문", "박웹")
]

# 초기 상태
print("초기 도서 목록:")
for book in books:
    print(f"  {book}")

# 대출 테스트
print("\n=== 대출 테스트 ===")
books[0].borrow("김학생", "admin123")  # 성공
books[1].borrow("이학생", "wrong_password")  # 실패 - 잘못된 비밀번호
books[1].borrow("이학생", "admin123")  # 성공
books[0].borrow("박학생", "admin123")  # 실패 - 이미 대출 중

# 현재 상태
print("\n대출 후 상태:")
for book in books:
    print(f"  {book}")
    print(f"    상세 정보: {book.get_info()}")

# 반납 테스트
print("\n=== 반납 테스트 ===")
books[0].return_book("admin123")  # 성공
books[2].return_book("admin123")  # 실패 - 대출되지 않은 도서

# 최종 상태
print("\n최종 상태:")
for book in books:
    print(f"  {book}")
    print(f"    대출 횟수: {book.borrow_count}회")

## 정리

### 캡슐화의 핵심 원칙:

#### 1. 데이터 은닉
- **비공개 속성** (`__`): 외부에서 직접 접근 불가
- **보호된 속성** (`_`): 관례상 내부 사용만
- **공개 속성**: 자유로운 외부 접근

#### 2. 접근 제어
- **Getter 메서드**: 안전한 데이터 조회
- **Setter 메서드**: 유효성 검사 포함 데이터 설정
- **@property 데코레이터**: 메서드를 속성처럼 사용

#### 3. 데이터 보호
- **유효성 검사**: 잘못된 값 설정 방지
- **타입 검사**: 올바른 데이터 타입 보장
- **비즈니스 로직**: 도메인 규칙 적용

#### 4. 인터페이스 설계
- **명확한 public 메서드**: 외부 사용을 위한 인터페이스
- **일관된 명명**: 의미 있는 메서드/속성 이름
- **문서화**: 사용법과 제약사항 명시

### 실제 적용에서의 이점:
1. **안전성**: 잘못된 데이터 입력 방지
2. **유지보수성**: 내부 구현 변경 시 외부 영향 최소화
3. **재사용성**: 안전한 인터페이스로 다양한 상황에서 활용
4. **디버깅**: 데이터 변경 지점 추적 용이

다음 장에서는 **다형성(Polymorphism)**에 대해 학습하겠습니다!